<a href="https://colab.research.google.com/github/matheusamorim/spark/blob/main/Contas_M%C3%A9dicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark #verificar se há o objeto spark 

In [7]:
df = spark.read.csv("/content/drive/My Drive/dados/GO AMB/2019_det_amb.csv", header=True, sep=",")


In [9]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType


In [46]:
def to_value(v):
    try:
        return float(v.replace(",","."))
    except:
        return 0.0
udf_to_value = F.udf(to_value, FloatType())

In [47]:
df= df.withColumn('VL_PROCEDIMENTO', udf_to_value(df["VL_PROCEDIMENTO"]))\
   .withColumn('QT_PROCEDIMENTO', udf_to_value(df["QT_PROCEDIMENTO"]))

In [ ]:
df.filter('VL_PROCEDIMENTO > "200.0"').show()

In [39]:
df.printSchema()

root
 |-- ID_EVENTO: string (nullable = true)
 |-- CD_TUSS_PROCEDIMENTO: string (nullable = true)
 |-- QT_PROCEDIMENTO: float (nullable = true)
 |-- VL_PROCEDIMENTO: float (nullable = true)
 |-- VL_PAGO_FORNECEDOR: string (nullable = true)
 |-- CD_TABELA_REFERENCIA: string (nullable = true)
 |-- LG_PACOTE: string (nullable = true)
 |-- IND_TABELA_PROPRIA: string (nullable = true)
 |-- DT_REALIZACAO: string (nullable = true)
 |-- UF_PRESTADOR: string (nullable = true)
 |-- LG_OUTLIER: string (nullable = true)

